In [1]:
from platform import python_version
print(python_version())

3.6.8


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Import data

In [3]:
# import datasets

forecast1 = pd.read_csv("Data/windforecasts_wf1.csv")
forecast2 = pd.read_csv("Data/windforecasts_wf2.csv")
forecast3 = pd.read_csv("Data/windforecasts_wf3.csv")
forecast4 = pd.read_csv("Data/windforecasts_wf4.csv")
forecast5 = pd.read_csv("Data/windforecasts_wf5.csv")
forecast6 = pd.read_csv("Data/windforecasts_wf6.csv")
forecast7 = pd.read_csv("Data/windforecasts_wf7.csv")
train = pd.read_csv("Data/train.csv")
benchmark = pd.read_csv("Data/benchmark.csv")

forecasts = [forecast1, forecast2, forecast3, forecast4, forecast5, forecast6, forecast7]
# Forecast structure: date hors u v ws wd
#train structure: date wp1 wp2 wp3 wp4 wp5 wp6 wp7

In [4]:
plot_titles = []
for i in range(7):
    plot_titles.append(f'Windfarm {i+1}')

plot_titles

['Windfarm 1',
 'Windfarm 2',
 'Windfarm 3',
 'Windfarm 4',
 'Windfarm 5',
 'Windfarm 6',
 'Windfarm 7']

# Functions

In [5]:
def plot_power_vs_direction(name, dir_series, wp_series):
    plt.scatter(dir_series, wp_series)
    plt.xlabel('Wind direction')
    plt.ylabel('Power production')
    plt.title(name)

In [6]:
# Function to remove overlapping forecasts from a forecast stored in a panda dataframe:

def remove_overlap(forecast_df):
    array = np.array([])
    r, c = forecast_df.shape
    for k in range(forecast_df.shape[1]):
        for i in range(0, forecast_df.shape[0], 48):
            for j in range(12):
                array = np.append(array, forecast_df.iloc[i+j, k])
    array2 = np.resize(array, (c, int(array.shape[0]/c)))
    fdict = {'Date': array2[0].astype(int), 'Hour': array2[1], 'u': array2[2], 'v': array2[3], 'wind_speed': array2[4], 'wind_dir': array2[5]}
    df = pd.DataFrame(data=fdict)
    return df

In [7]:
def convert_date_format(forecast_dataframe):
    for i in range (forecast_dataframe.shape[0]):
        # Fetch the date in its original format:
        prev_date = str(forecast_dataframe.iloc[i,0])
        forecast_hour = int(prev_date[-2:])
        
        # Fetch the hour the forecast is for:
        # FIX THIS: 1-12 INSTEAD OF 0-23
        hours_ahead = int(forecast_dataframe.iloc[i,1])
        hour = str(forecast_hour + hours_ahead).zfill(2)
        
        # Strip the hours the forecast was made
        date = prev_date[:-2]
        
        #append the hour the forecast is for at the end of the date:
        date_time = date + hour
        
        # Replace the date with the new date format:
        forecast_dataframe.iloc[i,0] = date_time
    
    return forecast_dataframe

# Cleaning

In [9]:
df = remove_overlap(forecast1)
df

,Date,Hour,u,v,wind_speed,wind_dir
0,2009070100,1.0,2.34,-0.79,2.47,108.68
1,2009070100,2.0,2.18,-0.99,2.40,114.31
2,2009070100,3.0,2.20,-1.21,2.51,118.71
3,2009070100,4.0,2.35,-1.40,2.73,120.86
4,2009070100,5.0,2.53,-1.47,2.93,120.13
5,2009070100,6.0,2.66,-1.29,2.96,115.79
6,2009070100,7.0,2.69,-0.81,2.81,106.71
7,2009070100,8.0,2.72,-0.26,2.73,95.39
8,2009070100,9.0,2.87,0.08,2.87,88.50
9,2009070100,10.0,3.23,-0.01,3.23,90.19


In [10]:
df = convert_date_format(df)

In [35]:
df['Date'] = df['Date'].apply(int)

In [36]:
train['date'] = train['date'].apply(int)

In [37]:
df.loc[df['Date'].isin(train['date'])]

,Date,Hour,u,v,wind_speed,wind_dir
0,2009070101,1.0,2.34,-0.79,2.47,108.68
1,2009070102,2.0,2.18,-0.99,2.40,114.31
2,2009070103,3.0,2.20,-1.21,2.51,118.71
3,2009070104,4.0,2.35,-1.40,2.73,120.86
4,2009070105,5.0,2.53,-1.47,2.93,120.13
5,2009070106,6.0,2.66,-1.29,2.96,115.79
6,2009070107,7.0,2.69,-0.81,2.81,106.71
7,2009070108,8.0,2.72,-0.26,2.73,95.39
8,2009070109,9.0,2.87,0.08,2.87,88.50
9,2009070110,10.0,3.23,-0.01,3.23,90.19


In [38]:
train.shape

(18757, 8)